# Data Analysis Project: Ecommerce Customer Behaviour

## Data Overview

<b> Calling libraries

In [9]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import sklearn.preprocessing 
import seaborn as sns
import kaggle
from zipfile import ZipFile

In [11]:
!kaggle datasets download -d shriyashjagtap/e-commerce-customer-for-behavior-analysis


  0%|          | 0.00/9.94M [00:00<?, ?B/s]
 10%|█         | 1.00M/9.94M [00:00<00:07, 1.19MB/s]
 20%|██        | 2.00M/9.94M [00:01<00:03, 2.22MB/s]
 30%|███       | 3.00M/9.94M [00:01<00:02, 3.05MB/s]
 40%|████      | 4.00M/9.94M [00:01<00:01, 3.88MB/s]
 50%|█████     | 5.00M/9.94M [00:01<00:01, 4.47MB/s]
 60%|██████    | 6.00M/9.94M [00:01<00:00, 4.63MB/s]
 70%|███████   | 7.00M/9.94M [00:01<00:00, 5.06MB/s]
 80%|████████  | 8.00M/9.94M [00:02<00:00, 5.08MB/s]
 91%|█████████ | 9.00M/9.94M [00:02<00:00, 5.03MB/s]
100%|██████████| 9.94M/9.94M [00:02<00:00, 5.00MB/s]
100%|██████████| 9.94M/9.94M [00:02<00:00, 4.03MB/s]
